<h2>Today Let's Make mobile document scanner</h2>
<br>
<h3>BASIC STEPS</h3>
<br>
<font color = "#547485">
<h6>1. Detect Edges <br> 2. Use the edges to find the contours to represent document being scanned
<br> 3. Apply perspective transform to obtain top-down view of the document</h6>
    </font>

In [3]:
#importing required libraries
from pyimagesearch.transform import four_point_transform
from skimage.filters import threshold_local
import numpy as np
import argparse
import cv2
import imutils
 


In [2]:
#constructing the argument parser
ap = argparse.ArgumentParser()
ap.add_argument("-i", "--image", required = True,
    #help = "Path to the image to be scanned")
args = vars(ap.parse_args())

SyntaxError: unexpected EOF while parsing (<ipython-input-2-0eb48f80c4e8>, line 5)

In [31]:
#loading the image from the arguments and compute the ratio of old height to new height
#cloning it and resizing it.
image = cv2.imread("doc1.jpeg") #later change this to args[image] in order to take image as argument from the terminal
ratio = image.shape[0]/500.0 #keeping track of the ratio of origional height to new height.. idk why.. let's see further
orig = image.copy()
image = imutils.resize(image,height = 500)



In [32]:
#converting image to grayscale, adding gaussian blur and taking edged image
gray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
blur = cv2.GaussianBlur(gray,(5,5),0)
edged = cv2.Canny(gray,50,90)

In [36]:
#show the origional image and the image after edge detection
print("STEP1: Edge Detection Done")
cv2.imshow("Origional Image",orig)
cv2.imshow("Image after Edge Detection",edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP1: Edge Detection Done


<h2>Now lets found Contours</h2>

In [34]:
#finding the contours in the edged image keeping on the largest edges and initializing the screen contours
cnts = cv2.findContours(edged.copy(),cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sorted(cnts, key = cv2.contourArea, reverse = True)[:5]

#loop for contours
for c in cnts:
    #approximating the contour
    peri = cv2.arcLength(c,True)
    approx = cv2.approxPolyDP(c,0.02*peri,True)
    
    #if our approximated contour has four points, then we can assume that we have found our screen
    if len(approx) == 4:
        screenCnt = approx
        break
        
#show the contour on the image
print("STEP2: Contour Done")
cv2.drawContours(image,[screenCnt], -1,(0,255,0),2)
cv2.imshow("Outline", image)
cv2.waitKey(0)
cv2.destroyAllWindows()

STEP2: Contour Done


In [35]:
#apply the four point transform to obtain a top-down view of origional image
warped = four_point_transform(orig, screenCnt.reshape(4,2)*ratio)

#convert the warped image to grayscale, and thresholding it to give it black and white paper effect
warped = cv2.cvtColor(warped, cv2.COLOR_BGR2GRAY)
T = threshold_local(warped,11,offset = 10, method = "gaussian")
warped = (warped > T).astype("uint8")*255

#show the origional and scanned images
print("STEP3 : FINALLY APPLIED PERSPECTIVE TRANSFORM")
cv2.imshow("Origional Image", imutils.resize(orig,height = 650))
cv2.imshow("Final Image", imutils.resize(warped,height = 650))
cv2.waitKey(0)
cv2.destroyAllWindows

STEP3 : FINALLY APPLIED PERSPECTIVE TRANSFORM


<function destroyAllWindows>